In [6]:
import pandas as pd
import torch
import copy
import xlrd
import numpy as np
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
 
batch_size = 14
# Data_Xlsx 
def Data_Reading(Normalization = True):
    # Read the xlsx
    train_x = pd.read_excel("trainingset.xlsx", 'Input',header = None)
    test_x = pd.read_excel("oilsset.xlsx",'oils',header = None)
    test_z = pd.read_excel("newodorset.xlsx",'new',header = None)
    train_y = pd.read_excel("trainingy1.xlsx",'Output',header = None)
    # Store the bounds
    train_x_bounds = [[train_x[col].max(), train_x[col].min()] for col in train_x]
    test_x_bounds = [[test_x[col].max(), test_x[col].min()] for col in test_x]
    test_z_bounds = [[test_z[col].max(), test_z[col].min()] for col in test_z]
    train_y_bounds = [[train_y[col].max(),train_y[col].min()] for col in train_y]
    # Normalization
    train_x_Normed = copy.deepcopy(train_x).apply(lambda x : (x - np.min(x)) / (np.max(x) - np.min(x)))
    test_x_Normed = copy.deepcopy(test_x).apply(lambda x : (x - np.min(x)) / (np.max(x) - np.min(x)))
    test_z_Normed = copy.deepcopy(test_z).apply(lambda x : (x - np.min(x)) / (np.max(x) - np.min(x)))
    train_y_Normed = copy.deepcopy(train_y).apply(lambda x : (x - np.min(x)) / (np.max(x) - np.min(x)))
    # xlsx to tensor
    if Normalization:
        train_x = torch.from_numpy(train_x_Normed.values).type(torch.FloatTensor)
        test_x = torch.from_numpy(test_x_Normed.values).type(torch.FloatTensor)
        test_z = torch.from_numpy(test_z_Normed.values).type(torch.FloatTensor)
        train_y = torch.from_numpy(train_y_Normed.values).type(torch.LongTensor)
    else:
        train_x = torch.from_numpy(train_x.values).type(torch.FloatTensor)
        test_x = torch.from_numpy(test_x.values).type(torch.FloatTensor)
        test_z = torch.from_numpy(test_z.values).type(torch.FloatTensor)
        train_y = torch.from_numpy(train_y.values).type(torch.LongTensor)
    #reshape
    train_x= train_x.view(238,1,16,1496)
    test_x= test_x.view(108,1,16,1496)
    test_z= test_z.view(95,1,16,1496)
    train_y=  train_y.squeeze()
    return train_x, test_x, test_z, train_y


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #input shape (1,16,1496)
        self.conv1 = nn.Conv2d(1,6,(3,5)) #con2d出来结果大小不变output shape (6,16,1496)
        self.conv2 = nn.Conv2d(6,10,(3,5)) 
        self.mp1 = nn.MaxPool2d(kernel_size=2, stride=2) 
        self.mp2 =  nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc = nn.Linear(10*2*370,10)   
             
    def forward(self, x):
        x = F.relu(self.mp1(self.conv1(x)))#（238,6,9，749）
        x = F.relu(self.mp2(self.conv2(x)))#（238,16,5，375）
        x = x.view(x.size(0), -1) # flatten the tensor
        x = self.fc(x)
        return x


cnn = Net()
print(cnn)

optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)
loss_func = nn.CrossEntropyLoss()

#training
def train(train_x,train_y):
    for epoch in range(5):
        t_x = Variable(train_x)
        t_y = Variable(train_y)
        #forward
        out = cnn(t_x)
        loss = loss_func(out, t_y)
        optimizer.zero_grad()
        #backward
        loss.backward()
        optimizer.step()
        print('Epoch[{}/{}], loss: {:.12f},'.format(epoch + 1,5, loss.item()))
            
#predicting 
def predict(test_x,test_z):
    for epoch in range(5):
        te_x = Variable(test_x)
        te_z = Variable(test_z)
        out1 = cnn(te_x)
        out2 = cnn(te_z)
        print('epoch',epoch+1,out1)
    

train_x, test_x, test_z, train_y = Data_Reading(Normalization = True)
train(train_x,train_y)
predict(test_x,test_z)



Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 5), stride=(1, 1))
  (conv2): Conv2d(6, 10, kernel_size=(3, 5), stride=(1, 1))
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (mp2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=7400, out_features=10, bias=True)
)
Epoch[1/5], loss: nan,
Epoch[2/5], loss: nan,
Epoch[3/5], loss: nan,
Epoch[4/5], loss: nan,
Epoch[5/5], loss: nan,
epoch 1 tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<ThAddmmBackward>)
epoch 2 tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],


In [2]:
train_x = pd.read_excel("trainingset.xlsx", 'Input',header = None)
train_x

,0,1,2,3,4,5,6,7,8,9,...,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495
0,-1.20000,-0.20000,-1.20000,-1.20000,-1.20000,-1.20000,-0.2000,-0.2000,2.8000,3.800,...,3.8000,4.8000,3.8000,4.8000,3.8000,4.8000,4.8000,4.8000,3.8000,4.8000
1,-2.33330,-0.33333,-1.33330,-1.33330,-2.33330,-2.33330,-1.3333,2.6667,8.6667,13.667,...,10.6670,11.6670,11.6670,11.6670,11.6670,11.6670,11.6670,10.6670,10.6670,10.6670
2,-3.62500,-3.62500,-3.62500,-3.62500,-3.62500,-3.62500,-0.6250,22.3750,46.3750,51.375,...,10.3750,10.3750,10.3750,10.3750,10.3750,10.3750,10.3750,10.3750,9.3750,10.3750
3,-0.85714,0.14286,-0.85714,-0.85714,-0.85714,-1.85710,5.1429,17.1430,24.1430,26.143,...,6.1429,5.1429,5.1429,6.1429,6.1429,5.1429,5.1429,6.1429,5.1429,5.1429
4,-2.57140,-1.57140,-1.57140,-1.57140,-1.57140,-1.57140,10.4290,26.4290,33.4290,35.429,...,21.4290,21.4290,21.4290,21.4290,21.4290,21.4290,21.4290,21.4290,21.4290,21.4290
5,-2.28570,-3.28570,-3.28570,-3.28570,-3.28570,-0.28571,15.7140,29.7140,34.7140,34.714,...,19.7140,19.7140,19.7140,19.7140,20.7140,19.7140,19.7140,19.7140,19.7140,20.7140
6,-3.00000,-3.00000,-4.00000,-3.00000,-3.00000,16.00000,58.0000,79.0000,84.0000,83.000,...,43.0000,44.0000,43.0000,44.0000,43.0000,44.0000,44.0000,43.0000,43.0000,44.0000
7,-1.33330,-1.33330,-1.33330,-1.33330,-0.33333,5.66670,12.6670,14.6670,14.6670,13.667,...,1.6667,1.6667,1.6667,1.6667,1.6667,1.6667,2.6667,2.6667,2.6667,2.6667
8,-1.50000,-1.50000,-0.50000,0.50000,0.50000,0.50000,0.5000,0.5000,0.5000,0.500,...,-19.5000,-20.5000,-20.5000,-20.5000,-20.5000,-20.5000,-20.5000,-19.5000,-20.5000,-19.5000
9,0.20000,-0.80000,0.20000,-0.80000,0.20000,0.20000,0.2000,0.2000,0.2000,0.200,...,-0.8000,-1.8000,-1.8000,-0.8000,-1.8000,-1.8000,-1.8000,-1.8000,-0.8000,-1.8000


In [3]:
test_x = pd.read_excel("oilsset.xlsx",'oils',header = None)
test_x

,0,1,2,3,4,5,6,7,8,9,...,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495
0,-18.6670,-17.66700,-17.66700,-17.66700,-16.66700,88.3330,481.3300,975.330,1430.300,1814.300,...,-17.66700,-16.66700,-16.66700,-16.66700,-17.66700,-16.66700,-16.66700,-16.66700,-16.66700,-16.66700
1,-3.6000,-3.60000,-3.60000,-3.60000,14.40000,527.4000,1711.4000,2870.400,3715.400,4194.400,...,-4.60000,-3.60000,-3.60000,-3.60000,-3.60000,-3.60000,-4.60000,-4.60000,-3.60000,-3.60000
2,-2.2000,-1.20000,-1.20000,-1.20000,5.80000,119.8000,355.8000,631.800,912.800,1164.800,...,-1.20000,-2.20000,-2.20000,-2.20000,-2.20000,-1.20000,-2.20000,-2.20000,-2.20000,-2.20000
3,-20.2000,-20.20000,-21.20000,-21.20000,82.80000,684.8000,1423.8000,2095.800,2694.800,3213.800,...,-21.20000,-21.20000,-21.20000,-21.20000,-21.20000,-21.20000,-21.20000,-21.20000,-21.20000,-21.20000
4,-71.0000,-71.00000,-71.00000,-69.00000,282.00000,2082.0000,4828.0000,7688.000,10300.000,12541.000,...,-74.00000,-75.00000,-75.00000,-74.00000,-75.00000,-75.00000,-74.00000,-74.00000,-75.00000,-74.00000
5,-1.5000,-1.50000,-1.50000,4.50000,249.50000,878.5000,1920.5000,3002.500,3791.500,4333.500,...,-11.50000,-12.50000,-11.50000,-12.50000,-12.50000,-11.50000,-11.50000,-12.50000,-11.50000,-12.50000
6,-64.2500,-63.25000,-64.25000,191.75000,3591.80000,7916.8000,11439.0000,14283.000,16574.000,18430.000,...,-75.25000,-75.25000,-75.25000,-75.25000,-75.25000,-75.25000,-75.25000,-75.25000,-75.25000,-75.25000
7,-137.0000,-137.00000,-137.00000,411.00000,2270.00000,3792.0000,4918.0000,5709.000,6241.000,6574.000,...,-135.00000,-136.00000,-136.00000,-136.00000,-135.00000,-135.00000,-136.00000,-135.00000,-136.00000,-135.00000
8,0.0000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.000,0.000,0.000,...,62.00000,62.00000,61.00000,61.00000,62.00000,61.00000,61.00000,62.00000,61.00000,61.00000
9,-0.3000,-0.30000,-1.30000,-1.30000,-0.30000,-0.3000,-0.3000,-0.300,0.700,3.700,...,-0.30000,-0.30000,-0.30000,-0.30000,-0.30000,-0.30000,-0.30000,-0.30000,-0.30000,-0.30000


In [4]:
test_z = pd.read_excel("newodorset.xlsx",'new',header = None)
test_z

,0,1,2,3,4,5,6,7,8,9,...,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495
0,-3.5000,-3.5000,-4.5000,-4.50000,-3.5000,19.5000,221.5000,627.50000,1104.500,1562.50,...,-5.5000,-4.5000,-5.50000,-4.50000,-5.50000,-4.5000,-5.50000,-5.5000,-4.5000,-5.50000
1,-11.0000,-12.0000,-11.0000,-11.00000,-11.0000,56.0000,348.0000,764.00000,1186.000,1568.00,...,-12.0000,-12.0000,-12.00000,-12.00000,-12.00000,-12.0000,-12.00000,-12.0000,-12.0000,-12.00000
2,-1.6667,-1.6667,-1.6667,-2.66670,-1.6667,9.3333,40.3330,93.33300,169.330,238.33,...,-2.6667,-2.6667,-2.66670,-2.66670,-2.66670,-2.6667,-3.66670,-2.6667,-3.6667,-3.66670
3,-3.6000,-3.6000,-2.6000,-3.60000,13.4000,274.4000,816.4000,1427.40000,2000.400,2505.40,...,-5.6000,-5.6000,-6.60000,-5.60000,-5.60000,-5.6000,-6.60000,-5.6000,-5.6000,-6.60000
4,-4.6000,-5.6000,-4.6000,-5.60000,20.4000,219.4000,603.4000,1066.40000,1524.400,1941.40,...,-6.6000,-6.6000,-6.60000,-6.60000,-7.60000,-6.6000,-6.60000,-6.6000,-6.6000,-6.60000
5,-9.6000,-9.6000,-9.6000,-9.60000,38.4000,244.4000,588.4000,1093.40000,1735.400,2420.40,...,-11.6000,-11.6000,-11.60000,-10.60000,-11.60000,-11.6000,-11.60000,-10.6000,-11.6000,-11.60000
6,-5.2500,-5.2500,-5.2500,15.75000,1007.8000,3577.8000,6303.8000,8724.80000,10785.000,12551.00,...,-5.2500,-5.2500,-5.25000,-5.25000,-5.25000,-6.2500,-6.25000,-5.2500,-5.2500,-5.25000
7,-9.2500,-9.2500,-8.2500,26.75000,266.7500,502.7500,689.7500,834.75000,942.750,1022.80,...,-10.2500,-10.2500,-10.25000,-10.25000,-10.25000,-10.2500,-10.25000,-10.2500,-10.2500,-10.25000
8,-0.1000,-0.1000,-0.1000,-0.10000,-0.1000,-0.1000,-0.1000,-0.10000,-0.100,0.90,...,42.9000,43.9000,43.90000,43.90000,42.90000,42.9000,42.90000,42.9000,42.9000,42.90000
9,-0.8000,-0.8000,-0.8000,-0.80000,-0.8000,-0.8000,-0.8000,-0.80000,0.200,6.20,...,4.2000,5.2000,5.20000,4.20000,4.20000,4.2000,4.20000,4.2000,4.2000,4.20000


In [5]:
train_y = pd.read_excel("trainingy1.xlsx",'Output',header = None)
train_y

,0
0,11721.7500
1,11721.7500
2,11721.7500
3,14998.7500
4,14998.7500
5,14998.7500
6,14998.7500
7,14998.7500
8,9532.2500
9,9532.2500
